In [2]:
!sudo pip install scikit-learn

[sudo] password for bentley: 



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import random

ImportError: No module named 'sklearn.model_selection'

Let's practice splitting up a multi-class into multiple binaries

In [ ]:
dat = pd.DataFrame({'T': [1, 2, 3, 4, 5]})
dat

In [ ]:
res = pd.get_dummies(dat, columns='T')
res

Now let's do it with our dataset

In [ ]:
train = pd.read_csv('data/flit-train.csv')
train

In [ ]:
# I now choose not to train on the name of the test
#train_res = pd.get_dummies(train, columns=['compiler', 'name', 'precision'])
train_res = pd.get_dummies(train, columns=['compiler', 'precision'])
train_res

In [ ]:
labels = pd.DataFrame(train_res['score0'])
data = train_res.drop('score0', axis=1)

In [ ]:
# This did not split on boundaries of tests well.
# Meaning the training set and test set both had
# instances of the same FLiT test name.  We want
# it to generalize to new tests never before seen.
#train_dat, test_dat, train_lab, test_lab = \
#    train_test_split(data, labels, train_size=0.8, random_state=0)
names = set(data['name'])
train_names = random.sample(names, int(0.8 * len(names)))
train_selector = data.name.apply(lambda x: x in train_names)
data = data.drop('name', 1)
train_dat = data[train_selector]
train_lab = labels[train_selector]
test_selector = train_selector.apply(lambda x: not x)
test_dat = data[test_selector]
test_lab = labels[test_selector]
print(len(train_dat), len(test_dat))

In [ ]:
plt.hist(train_lab.values.ravel())
plt.show()

In [ ]:
print('Train Percent of ones: ',
      sum(train_lab.values.ravel()) / len(train_lab))
print('Test  Percent of ones: ',
      sum(test_lab.values.ravel()) / len(test_lab))

In [ ]:
clf = svm.SVC()
clf.fit(train_dat, train_lab.values.ravel())

In [ ]:
clf.score(test_dat, test_lab)

In [ ]:
test_pred = clf.predict(test_dat)

In [ ]:
print('Number of affirmative predictions: ', sum(test_pred))
print('Size of the test set:              ', len(test_pred))

In [ ]:
train_pred = clf.predict(train_dat)

In [ ]:
print('Number of affirmative predictions: ', sum(train_pred))
print('Size of the train set:             ', len(train_pred))

In [ ]:
width = 0.4
fig, ax = plt.subplots()

true_pos = lambda p, l: sum((p == 1) & (l == 1))
true_neg = lambda p, l: sum((p == 0) & (l == 0))
false_pos = lambda p, l: sum((p == 1) & (l == 0))
false_neg = lambda p, l: sum((p == 0) & (l == 1))

def vals(p, l):
    v = np.asarray([
            true_pos(p, l),
            false_neg(p, l),
            true_neg(p, l),
            false_pos(p, l),
            ], dtype=np.double)
    # Normalize what should be positive examples
    v[:2] /= sum(v[:2])
    # Normalize what should be negative examples
    v[2:] /= sum(v[2:])
    return v

train_vals = vals(train_pred, train_lab.score0)
test_vals = vals(test_pred, test_lab.score0)

ind = np.arange(4)
width = 0.3
train_bar = ax.bar(ind, train_vals*100, width, color='#2b8cbe')
test_bar = ax.bar(ind + width, test_vals*100, width, color='#a6bddb')

ax.set_ylabel('Percent')
ax.set_title('Accuracy of default SVM on FLiT data')
ax.set_xticks(ind + width/2)
ax.set_xticklabels(('True Pos', 'False Neg', 'True Neg', 'False Pos'))
ax.legend((train_bar[0], test_bar[0]), ('Train', 'Test'))

plt.show()

This looks pretty bad.  The SVM simply learned to classify EVERYTHING as positive examples.

Let's try something.  Maybe I can duplicate the negative examples so that it's more than 7% of the training set.

Another way to approach this would be to change the scoring function.  We would choose not necessarily to optimize the total precision, but rather we want to optimize the average of the separate class precisions.  I don't know how I would do this in sklearn, so I'll go with my first idea for now.  

Note: I could try to do this if I were to implement it myself instead of using sklearn.

In [ ]:
#[i for i in range(len(train_lab)) if train_lab[i] == 0]
#train_lab[train_lab.score0 == 1]
print(sum(train_lab.values.ravel()), len(train_lab.values.ravel()))
print(sum(test_lab.values.ravel()), len(test_lab.values.ravel()))

In [ ]:
only_zero_dat = train_dat[train_lab.score0 == 0]
only_zero_lab = train_lab[train_lab.score0 == 0]
only_zero_dat

In [ ]:
big_dat = train_dat
big_lab = train_lab
for i in range(10):
    big_dat = pd.concat([big_dat, only_zero_dat])
    big_lab = pd.concat([big_lab, only_zero_lab])

In [ ]:
print(len(big_lab), sum(big_lab.values.ravel()))

In [ ]:
big_clf = svm.SVC()
big_clf.fit(big_dat, big_lab.values.ravel())

In [ ]:
big_clf.score(big_dat, big_lab)

In [ ]:
big_pred = big_clf.predict(big_dat)

In [ ]:
print(sum(big_pred), '/', len(big_pred))

In [ ]:
big_clf.score(test_dat, test_lab)

In [ ]:
big_clf.score(train_dat, train_lab)

In [ ]:
train_pred_from_big = big_clf.predict(train_dat)
print(sum(train_pred_from_big), '/', len(train_pred_from_big))

In [ ]:
test_pred_from_big = big_clf.predict(test_dat)

In [ ]:
print(sum(test_pred_from_big), '/', len(test_pred_from_big))

In [ ]:
print(sum(test_lab.values.ravel()), '/', len(test_lab))

In [ ]:
sum((train_pred_from_big == 1) & (train_lab.score0 == 1))

In [ ]:
width = 0.4
fig, ax = plt.subplots()

true_pos = lambda p, l: sum((p == 1) & (l == 1))
true_neg = lambda p, l: sum((p == 0) & (l == 0))
false_pos = lambda p, l: sum((p == 1) & (l == 0))
false_neg = lambda p, l: sum((p == 0) & (l == 1))

def vals(p, l):
    v = np.asarray([
            true_pos(p, l),
            false_neg(p, l),
            true_neg(p, l),
            false_pos(p, l),
            ], dtype=np.double)
    # Normalize what should be positive examples
    v[:2] /= sum(v[:2])
    # Normalize what should be negative examples
    v[2:] /= sum(v[2:])
    return v

big_vals = vals(big_pred, big_lab.score0)
train_vals = vals(train_pred_from_big, train_lab.score0)
test_vals = vals(test_pred_from_big, test_lab.score0)

ind = np.arange(4)
width = 0.3
train_bar = ax.bar(ind, train_vals*100, width, color='#2b8cbe')
test_bar = ax.bar(ind + width, test_vals*100, width, color='#a6bddb')

ax.set_ylabel('Percent')
ax.set_title('Accuracy of default SVM on FLiT data')
ax.set_xticks(ind + width/2)
ax.set_xticklabels(('True Pos', 'False Neg', 'True Neg', 'False Pos'))
ax.legend((train_bar[0], test_bar[0]), ('Train', 'Test'))

plt.show()

Note: The counts for the examples that are supposed to be positive (i.e. 'True Pos' and 'False Neg') are normalized separately from the examples that should be negative (i.e. 'True Neg' and 'False Pos').  I wanted to get percentages of accuracy in each class.

In [ ]:
print(big_vals)
print(train_vals)
print(test_vals)

I wonder what the classifier found to be good features to use.

In [ ]:
clf.get_params()

**TODO:** Find the coefficients of the SVM to see what it learned

**TODO:** Do cross-validation with different kernels and different C values

There is code here: http://www.eric-kim.net/eric-kim-net/posts/1/kernel_trick.html

This code does some cross validation using the sklearn module.  It would be good to become much more familiar with this module.

In [ ]:
print(classification_report(test_lab, test_pred_from_big))